### some standard imports



In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import wbdata
import pandas as pd
import datetime
import os


In [7]:
#  load actual gdp data 

def get_gdp_data():
    # Define the date range (past 35 years)
    start_date = datetime.datetime(1989, 1, 1)
    end_date = datetime.datetime(2024, 1, 1)

    # Define the top 25 economies by ISO country codes (based on search results)
    countries = [
        'US', 'CN', 'JP', 'DE', 'IN', 'GB', 'FR', 'BR', 'IT', 'CA',
        'KR', 'RU', 'AU', 'ES', 'MX', 'ID', 'NL', 'SA', 'TR', 'CH',
        'TW', 'PL', 'SE', 'TH', 'BE'
    ]

    # Define the GDP indicator (current US$)
    indicator = {'NY.GDP.MKTP.CD': 'GDP (current US$)'}

    # Retrieve GDP data from the World Bank API
    df = wbdata.get_dataframe(indicator, country=countries, date=(start_date, end_date), 
                            #   convert_date=False
                            )

    # Reshape the data for better readability
    df = df.reset_index()
    df_pivot = df.pivot(index='date', columns='country', values='GDP (current US$)')

    return df_pivot



output_folder = os.path.join('..', '..', "_data", "gdp_data")
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Save the GDP data to a CSV file in the created subfolder
output_file = os.path.join(output_folder, "top_25_economies_gdp_1989_2024.csv")
if os.path.exists(output_file):
    gdp_df = pd.read_csv(output_file)
    print(f"GDP data loaded from {output_file}")
else:
    gdp_df = get_gdp_data()
    gdp_df.to_csv(output_file)
    print(f"GDP data saved to {output_file}")


GDP data loaded from ../../_data/gdp_data/top_25_economies_gdp_1989_2024.csv


### plot the actual chart


In [8]:
# df = pd.DataFrame(data)

# Melt the DataFrame for Plotly compatibility
df_melted = gdp_df.melt(id_vars=["date"], var_name="Country", value_name="GDP")

# # Sort the data dynamically by GDP within each year
# def reorder_categories(frame):
#     sorted_countries = frame.sort_values(by="GDP", ascending=False)["Country"].tolist()
#     frame["Country"] = pd.Categorical(frame["Country"], categories=sorted_countries, ordered=True)
#     return frame

# df_melted = df_melted.groupby("Year", group_keys=False).apply(reorder_categories)

# Create the Plotly animation
fig = px.bar(
    df_melted,
    x="Country",
    y="GDP",
    color="Country",
    animation_frame="date",
    title="Top 5 Economies Over Time",
    labels={"GDP": "GDP (Trillions USD)"},
    range_y=[0, gdp_df.drop('date', axis=1).max().max() * 1.1],
)

# Update layout for dynamic sorting of x-axis
fig.update_layout(
    xaxis_title="Country",
    yaxis_title="GDP (Trillions USD)",
    legend_title="Country",
    template="plotly_white",
    xaxis=dict(categoryorder="total descending"), # used for dynamically rearranging x axis
    xaxis_tickangle=45,  # Tilt the x-axis labels
)

# Show the interactive animation
fig.show()

In [9]:
fig.write_html('us_markets_animation.html', include_plotlyjs='cdn')